In [75]:
from trainer import UncertaintyTrainer
import os
import numpy as np
import pandas
import torch
from module.config import loadConfig, showConfig
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from torchvision import transforms
from module.utils import accuracy
from sklearn.manifold import TSNE
from module.config import checkOutputDirectoryAndCreate
gpu = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
# legend_items = list(range(0, 10))

legend_items = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship', 'truck']
config_path = "./result/cifar_noise_sym05_ssl_reg_blur_b128/config.yaml"

epoch = 199
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() and gpu != "-1" else "cpu")
config = loadConfig(config_path)
config['noise_type'] = "none"
trainer = UncertaintyTrainer(config, device)
checkOutputDirectoryAndCreate(f'{config["output_folder"]}/img')

model_path = f'result/{config["output_folder"]}/flow_{str(epoch).zfill(2)}.pt'
encoder_path = f'result/{config["output_folder"]}/encoder_{str(epoch).zfill(2)}.pt'
trainer.load(model_path)
trainer.load_encoder(encoder_path)



Files already downloaded and verified
Files already downloaded and verified
Training T : 1.0
Number of trainable parameters of Point CNF: 61265


In [54]:
MC_sample = 1
mean = 0
std = 0

probs_val, target_val, _, probs_all_val, condition_feature_val  = trainer.sampling(trainer.val_loader, MC_sample, mean, std)
acc_list = accuracy(probs_val,  target_val, topk=(1,2))
acc_top1 = acc_list[0].cpu().item()
acc_top2 = acc_list[1].cpu().item()
print("acc top1: ", acc_top1, " top2 : ", acc_top2)

78it [00:08,  9.20it/s]

acc top1:  81.76081848144531  top2 :  90.47476196289062


In [58]:
X_embedded = TSNE(n_components=2, verbose=1, init='random').fit_transform(x.numpy())

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.003s...
[t-SNE] Computed neighbors for 9984 samples in 2.409s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 1.461608
[t-SNE] KL divergence after 250 iterations with early exaggeration: 82.836823
[t-SNE] KL divergence after 1000 iterations: 1.567081


In [50]:
import seaborn as sns
import matplotlib.patheffects as PathEffects

sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, legend_items[i], fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
for epoch in range(0, 200, 10):
    model_path = f'result/{config["output_folder"]}/flow_{str(epoch).zfill(2)}.pt'
    encoder_path = f'result/{config["output_folder"]}/encoder_{str(epoch).zfill(2)}.pt'
    trainer.load(model_path)
    trainer.load_encoder(encoder_path)
    _, target_val, _, _, condition_feature_val  = trainer.sampling(trainer.val_loader, MC_sample, mean, std)
    X_embedded = TSNE(n_components=2, verbose=1, init='random').fit_transform(condition_feature_val.numpy())
    scatter(X_embedded, target_val.numpy())
    path = f'./result/{config["output_folder"]}/img/digits_tsne-generated_{epoch}.png'
    plt.savefig(path, dpi=120)

78it [00:06, 12.55it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.001s...
[t-SNE] Computed neighbors for 9984 samples in 2.769s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 1.039951
[t-SNE] KL divergence after 250 iterations with early exaggeration: 87.744255
[t-SNE] KL divergence after 1000 iterations: 1.955247


78it [00:06, 12.30it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.001s...
[t-SNE] Computed neighbors for 9984 samples in 2.733s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 2.455336
[t-SNE] KL divergence after 250 iterations with early exaggeration: 89.325874
[t-SNE] KL divergence after 1000 iterations: 2.067517


78it [00:06, 12.29it/s]


...

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.001s...
[t-SNE] Computed neighbors for 9984 samples in 2.515s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 1.922931
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.512871
[t-SNE] KL divergence after 1000 iterations: 1.617695


78it [00:08,  9.59it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.001s...
[t-SNE] Computed neighbors for 9984 samples in 2.665s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 1.786858
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.595497
[t-SNE] KL divergence after 1000 iterations: 1.625154


78it [00:07,  9.95it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9984 samples in 0.001s...
[t-SNE] Computed neighbors for 9984 samples in 2.520s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9984
[t-SNE] Computed conditional probabilities for sample 2000 / 9984
[t-SNE] Computed conditional probabilities for sample 3000 / 9984
[t-SNE] Computed conditional probabilities for sample 4000 / 9984
[t-SNE] Computed conditional probabilities for sample 5000 / 9984
[t-SNE] Computed conditional probabilities for sample 6000 / 9984
[t-SNE] Computed conditional probabilities for sample 7000 / 9984
[t-SNE] Computed conditional probabilities for sample 8000 / 9984
[t-SNE] Computed conditional probabilities for sample 9000 / 9984
[t-SNE] Computed conditional probabilities for sample 9984 / 9984
[t-SNE] Mean sigma: 1.745502
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.891327


# read folder file list

In [ ]:
from os import walk

filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file

# image to git

In [ ]:
import imageio
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('/path/to/movie.gif', images)